# Extrait le top 10 des chansons écoutées sur Deezer

# Export

In [61]:
import sys
from pathlib import Path

notebook_dir = Path().resolve()
project_root = notebook_dir.parent

sys.path.insert(0, str(project_root / 'src'))
print(f"✅ Chemin src ajouté : {project_root / 'src'}")

✅ Chemin src ajouté : /home/laurent/code/Laurent-Pa/epsi/AtelierIntegrationDonnee/Music-Analysis-ETL/src


In [62]:
import time

def get_album_genre(album_id):
    """Récupère le genre_id d'un album"""
    try:
        url = f"https://api.deezer.com/album/{album_id}"
        response = requests.get(url)
        if response.status_code == 200:
            album_data = response.json()
            # Les genres sont dans un tableau, on prend le premier
            genres = album_data.get('genres', {}).get('data', [])
            if genres:
                return genres[0].get('id', None)
        return None
    except Exception as e:
        print(f"Erreur album {album_id}: {e}")
        return None

def get_genre_name(genre_id):
    """Récupère le nom d'un genre à partir de son ID"""
    if genre_id is None:
        return None
    try:
        url = f"https://api.deezer.com/genre/{genre_id}"
        response = requests.get(url)
        if response.status_code == 200:
            genre_data = response.json()
            return genre_data.get('name', None)
        return None
    except Exception as e:
        print(f"Erreur genre {genre_id}: {e}")
        return None


In [63]:
import requests
import json

# Récupérer les données du chart Deezer
url = "https://api.deezer.com/chart"
response = requests.get(url)

# Vérifier que la requête a réussi
if response.status_code == 200:
    data = response.json()
    print("✅ Données de l'API Deeze Chart récupérées avec succès!")
    print(f"Clés disponibles : {data.keys()}")
else:
    print(f"❌ Erreur lors de la récupération de Chart de l'API Deezer: {response.status_code}")

# Afficher la structure
print("\n--- Structure des données ---")
for key in data.keys():
    if isinstance(data[key], dict) and 'data' in data[key]:
        print(f"{key}: {len(data[key]['data'])} éléments")

✅ Données de l'API Deeze Chart récupérées avec succès!
Clés disponibles : dict_keys(['tracks', 'albums', 'artists', 'playlists', 'podcasts'])

--- Structure des données ---
tracks: 10 éléments
albums: 10 éléments
artists: 10 éléments
playlists: 10 éléments
podcasts: 10 éléments


In [70]:
import pandas as pd

# Extraire les tracks
tracks_data = data['tracks']['data']

# Créer le DataFrame
df_tracks = pd.DataFrame(tracks_data)

# Afficher les premières lignes
print(df_tracks.head())

# Afficher les colonnes disponibles
print("\n--- Colonnes disponibles ---")
print(df_tracks.columns.tolist())

# Afficher les dimensions
print(f"\n📊 Dimensions : {df_tracks.shape}")

           id               title         title_short title_version  \
0  3558373981           melodrama           melodrama                 
1  3412534581              Golden              Golden                 
2  3651473412         Nostalgique         Nostalgique                 
3  3602108062            MON BÉBÉ            MON BÉBÉ                 
4  3538842341  Un monde à l'autre  Un monde à l'autre                 

                                      link  duration    rank  explicit_lyrics  \
0  https://www.deezer.com/track/3558373981       176  996934            False   
1  https://www.deezer.com/track/3412534581       192  979012            False   
2  https://www.deezer.com/track/3651473412       207  974216             True   
3  https://www.deezer.com/track/3602108062       130  997284            False   
4  https://www.deezer.com/track/3538842341       155  991842             True   

   explicit_content_lyrics  explicit_content_cover  \
0                        0      

In [64]:
# On normalise les données pour sortir les objets imbriqués (artiste et album)
df_tracks_normalized = pd.json_normalize(tracks_data)
print(df_tracks_normalized.columns.tolist())

['id', 'title', 'title_short', 'title_version', 'link', 'duration', 'rank', 'explicit_lyrics', 'explicit_content_lyrics', 'explicit_content_cover', 'preview', 'md5_image', 'position', 'type', 'artist.id', 'artist.name', 'artist.link', 'artist.picture', 'artist.picture_small', 'artist.picture_medium', 'artist.picture_big', 'artist.picture_xl', 'artist.radio', 'artist.tracklist', 'artist.type', 'album.id', 'album.title', 'album.cover', 'album.cover_small', 'album.cover_medium', 'album.cover_big', 'album.cover_xl', 'album.md5_image', 'album.tracklist', 'album.type']


In [65]:
# Sélectionne les colonnes à conserver
df_tracks_filtered = df_tracks_normalized[['title','artist.name', 'artist.picture', 'album.id', 'explicit_lyrics']]
print(df_tracks_filtered)

# Vérifier les dimensions
print(f"\n📊 Dimensions : {df_tracks_filtered.shape}")
print(f"✅ {len(df_tracks_filtered)} tracks avec {len(df_tracks_filtered.columns)} colonnes")

                title  artist.name  \
0           melodrama        disiz   
1              Golden      HUNTR/X   
2         Nostalgique          Jul   
3            MON BÉBÉ        RnBoi   
4  Un monde à l'autre  GP Explorer   
5          PARISIENNE         GIMS   
6         Ballon D'or     Le Crime   
7           TP sur TP          Jul   
8  Génération impolie    Franglish   
9             Skyfall        Adele   

                                  artist.picture   album.id  explicit_lyrics  
0        https://api.deezer.com/artist/292/image  822345001            False  
1  https://api.deezer.com/artist/330311461/image  771853201            False  
2    https://api.deezer.com/artist/1191615/image  855003242             True  
3  https://api.deezer.com/artist/172662567/image  837893162            False  
4  https://api.deezer.com/artist/344313291/image  815406321             True  
5    https://api.deezer.com/artist/4429712/image  796651651            False  
6   https://api.deezer.com/a

In [69]:
print("🔍 Récupération des genres des albums...")
df_tracks_filtered['genre_id'] = df_tracks_filtered['album.id'].apply(
    lambda album_id: get_album_genre(album_id)
)
print("🎵 Récupération des noms de genres...")
df_tracks_filtered['genre_name'] = df_tracks_filtered['genre_id'].apply(
    lambda genre_id: get_genre_name(genre_id)
)

df_tracks_final = df_tracks_filtered[['title', 'artist.name', 'genre_name', 'explicit_lyrics']].copy()
df_tracks_final.columns = ['track', 'artist', 'genre', 'is_explicit_lyrics']
display(df_tracks_final.head())

# Convertir en JSON
result = df_tracks_final[['track', 'artist', 'genre', 'is_explicit_lyrics']].to_json(orient='records', force_ascii=False)
print(result)

🔍 Récupération des genres des albums...


/tmp/ipykernel_31391/1216971879.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tracks_filtered['genre_id'] = df_tracks_filtered['album.id'].apply(


🎵 Récupération des noms de genres...


/tmp/ipykernel_31391/1216971879.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tracks_filtered['genre_name'] = df_tracks_filtered['genre_id'].apply(


,track,artist,genre,is_explicit_lyrics
0,melodrama,disiz,Pop,False
1,Golden,HUNTR/X,Musique asiatique,False
2,Nostalgique,Jul,Rap/Hip Hop,True
3,MON BÉBÉ,RnBoi,R&B,False
4,Un monde à l'autre,GP Explorer,Rap/Hip Hop,True


[{"track":"melodrama","artist":"disiz","genre":"Pop","is_explicit_lyrics":false},{"track":"Golden","artist":"HUNTR\/X","genre":"Musique asiatique","is_explicit_lyrics":false},{"track":"Nostalgique","artist":"Jul","genre":"Rap\/Hip Hop","is_explicit_lyrics":true},{"track":"MON BÉBÉ","artist":"RnBoi","genre":"R&B","is_explicit_lyrics":false},{"track":"Un monde à l'autre","artist":"GP Explorer","genre":"Rap\/Hip Hop","is_explicit_lyrics":true},{"track":"PARISIENNE","artist":"GIMS","genre":"Rap\/Hip Hop","is_explicit_lyrics":false},{"track":"Ballon D'or","artist":"Le Crime","genre":"Rap\/Hip Hop","is_explicit_lyrics":true},{"track":"TP sur TP","artist":"Jul","genre":"Rap\/Hip Hop","is_explicit_lyrics":true},{"track":"Génération impolie","artist":"Franglish","genre":"Rap\/Hip Hop","is_explicit_lyrics":false},{"track":"Skyfall","artist":"Adele","genre":"Alternative","is_explicit_lyrics":false}]


# Vérification des blocs de code exportés vers l'appli

In [67]:
from extractors.extractor_deezer_chart import extract_deezer_chart

# Tester l'extraction
data = extract_deezer_chart()

✅ Données de l'API Deezer Chart récupérées avec succès!
Clés disponibles : dict_keys(['tracks', 'albums', 'artists', 'playlists', 'podcasts'])


In [71]:
from transformers.transformer_deezer_chart import transform_deezer_chart

# Tester la transformation
result = transform_deezer_chart(data)
print(f"\nNombre de tracks : {len(result)}")
print(f"Premier résultat : {result[0]}")

📊 10 tracks à traiter
🔍 Récupération des genres pour 10 albums uniques...
🎵 Récupération des noms pour 5 genres uniques...

Nombre de tracks : 10
Premier résultat : {'track': 'melodrama', 'artist': 'disiz', 'genre': 'Pop', 'is_explicit_lyrics': False}
